In [45]:
import cv2
import numpy as np

# Definiciones de constantes
FICHERO_PESOS = "C:/Users/LUCIA/Documents/material/pesos/face_detection_yunet_2023mar.onnx"
CONF_THRESHOLD = 0.30  # Umbral de confianza
NMS_THRESHOLD = 0.30    # Umbral de supresión no máxima
TOP_K = 5000            # Número máximo de detecciones
DIMX, DIMY = 320, 320   # Dimensiones de la imagen
IMAGEN = "C:/Users/LUCIA/Documents/material/imagenes/largest_selfie.jpg"

# Definición de colores en BGR
AZUL = (255, 0, 0)
ROJO = (0, 0, 255)
VERDE = (0, 255, 0)
MORADO = (255, 0, 255)
AMARILLO = (0, 255, 255)

# Inicializar el detector de caras
detector = cv2.FaceDetectorYN.create(
    FICHERO_PESOS, "", (DIMX, DIMY), CONF_THRESHOLD, NMS_THRESHOLD, TOP_K
)

# Cargar la imagen
image = cv2.imread(IMAGEN)
if image is None:
    print('Problema al cargar la imagen')
    exit()

# Configurar el tamaño de entrada y detectar caras
detector.setInputSize((image.shape[1], image.shape[0]))
faces = detector.detect(image)[1]

# Procesar las detecciones
if faces is not None and len(faces) > 0:
    for face in faces:
        # Dibujar el cuadro delimitador
        cv2.rectangle(
            image,
            (int(face[0]), int(face[1])),
            (int(face[0] + face[2]), int(face[1] + face[3])),
            VERDE, 2
        )
        # Dibujar los puntos clave (ojos, nariz, boca)
        cv2.circle(image, (int(face[4]), int(face[5])), 2, AZUL, -1)
        cv2.circle(image, (int(face[6]), int(face[7])), 2, ROJO, -1)
        cv2.circle(image, (int(face[8]), int(face[9])), 2, VERDE, -1)
        cv2.circle(image, (int(face[10]), int(face[11])), 2, MORADO, -1)
        cv2.circle(image, (int(face[12]), int(face[13])), 2, AMARILLO, -1)
        # Mostrar el porcentaje de confianza
        cv2.putText(
            image, f'{int(100 * face[14])}%',
            (int(face[0]), int(face[1]) + 15),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, VERDE, 1
        )
else:
    print('No se ha detectado ninguna cara')

# Mostrar y guardar la imagen resultante
cv2.imshow('image', image)
cv2.imwrite('DeteccionCaras.jpg', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [48]:
import cv2
import numpy as np

# Definición de archivos y parámetros
FICHERO_PESOS_DETECCION = 'C:/Users/LUCIA/Documents/material/pesos/face_detection_yunet_2023mar.onnx'
CONF_THRESHOLD_DETECCION = 0.90
NMS_THRESHOLD_DETECCION = 0.3
TOP_K = 5000

FICHERO_PESOS_RECONOCIMIENTO = 'C:/Users/LUCIA/Documents/material/pesos/face_recognition_sface_2021dec.onnx'

DIMX = 320
DIMY = 320

FICHERO_USUARIOS = 'C:/Users/LUCIA/Documents/material/usuarios/usuarios.txt'
FICHERO_FEATURES = 'C:/Users/LUCIA/Documents/material/usuarios/fichero_features_usuarios.txt'
NUM_FEATURES = 128

# Cargar usuarios y sus características faciales
usuarios = []
caracteristicas = []

with open(FICHERO_FEATURES, 'r') as file:
    lineas = file.readlines()
    for i in range(0, len(lineas), NUM_FEATURES):
        feature_vector = np.array([float(x.strip()) for x in lineas[i:i+NUM_FEATURES]], dtype=np.float32)
        caracteristicas.append(feature_vector)

with open(FICHERO_USUARIOS, 'r') as file:
    for linea in file:
        valores = linea.strip().split(',')
        if len(valores) == 2:
            usuarios.append(valores[1])

print(f'Número de usuarios registrados: {len(usuarios)}')

# Inicializar detector y reconocedor facial
detector = cv2.FaceDetectorYN.create(
    FICHERO_PESOS_DETECCION, '', (DIMX, DIMY),
    CONF_THRESHOLD_DETECCION, NMS_THRESHOLD_DETECCION, TOP_K
)

face_recognizer = cv2.FaceRecognizerSF.create(FICHERO_PESOS_RECONOCIMIENTO, '')

# Ruta de la imagen a reconocer
IMAGEN = "C:/Users/LUCIA/Documents/material/imagenes/imagenGrupo2.jpeg"
image = cv2.imread(IMAGEN)

if image is None:
    print(f'Error: No se pudo cargar la imagen {IMAGEN}')
else:
    detector.setInputSize(image.shape[1::-1])
    faces = detector.detect(image)[1]

    if faces is not None and len(faces) > 0:
        for i, face in enumerate(faces):
            x, y, w, h = map(int, face[:4])  # Coordenadas del rectángulo de la cara
            
            aligned_face = face_recognizer.alignCrop(image, face)
            feature = face_recognizer.feature(aligned_face).flatten()

            # Comparación con usuarios registrados
            min_dist = float('inf')
            usuario_reconocido = "Desconocido"

            for j, user_feature in enumerate(caracteristicas):
                distancia = np.linalg.norm(feature - user_feature)
                if distancia < min_dist:
                    min_dist = distancia
                    usuario_reconocido = usuarios[j]

            print(f'Cara {i+1}: {usuario_reconocido} (Distancia: {min_dist:.4f})')

            # Dibujar rectángulo alrededor de la cara
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, usuario_reconocido, (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

    else:
        print('Error: No se detectó ninguna cara en la imagen.')

    # Mostrar la imagen con las caras detectadas
    cv2.imshow('Reconocimiento Facial', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Número de usuarios registrados: 3
Cara 1:  Arturo E (Distancia: 9.6146)
Cara 2:  Jose Maria A (Distancia: 15.1816)
